#### Librerias Necesarias

In [1]:
(require '[clojure.string :as s])
(defn toInt[x] (Integer/parseInt x));Java
(defn toDouble[x] (Double/parseDouble x));Java

#'user/toDouble

Clase de Pelicula

In [2]:
(defrecord Movie [id title year age imdb rottenTomato netflix hulu primeVideo disneyPlus  director geners country lenguage runtime]
    Object 
    (toString [this]
              (format "%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s" id title year age imdb rottenTomato netflix hulu primeVideo disneyPlus director geners country lenguage runtime)
              )
    )

user.Movie

Lectura de informacion

In [3]:
(def data (vec (rest (s/split-lines (slurp "MoviesOnStreamingPlatforms_updated.csv") ) ) ) )

#'user/data

Conversion de datos

In [4]:
(defn toMovie[ record ]
    (Movie. 
     (toInt(get record 0))  
     (get record 1)  
     (toInt (get record 2)) 
     (get record 3) (toDouble (get record 4)) (toDouble (get record 5)) 
                                (toInt (get record 6)) 
                                (toInt (get record 7))
                                (toInt (get record 8))
                                (toInt (get record 9))
                                (get record 10)
                                (get record 11)
                                (get record 12)
                                (get record 13)
                                (toInt (get record 14)) 
                               ))

#'user/toMovie

In [5]:
(defn get-Movies-lazy [datos] 
    (lazy-seq (cons (toMovie (s/split (first datos) #","))(get-Movies-lazy (rest datos))) )
    )

#'user/get-Movies-lazy

In [6]:
(defn get-Vector-Valor [datos, valor]
    (lazy-seq (cons (valor (first datos))(get-Vector-Valor (rest datos) valor)))
    )

#'user/get-Vector-Valor

In [7]:
(def movies (take 20 (get-Movies-lazy data)))

#'user/movies

In [8]:
(def imdb (vec (take 20 (get-Vector-Valor movies :imdb))))

#'user/imdb

In [9]:
(def rottonTomato (vec (take 20 (get-Vector-Valor movies :rottonTomato))))

#'user/rottonTomato

In [10]:
(def runtime (vec (take 20 (get-Vector-Valor movies :runtime))))

#'user/runtime

In [11]:
(def x0 (vec (take 20 (repeat 1))))

#'user/x0

In [12]:
(def matrixA [x0 imdb runtime])

#'user/matrixA

Transpuesta

In [73]:
(defn get-New-Col [matrix]
    (def newCol [(first (get matrix 0))(first (get matrix 1)) (first (get matrix 2))])
    newCol
    )

#'user/get-New-Col

In [119]:
(defn transpuesta [matrix]
    (loop [mResult (atom '()) mTemp matrix] 
         (if (= 0 (count (get mTemp 1)))
             mResult
             (do
                 (def m (get-New-Col mTemp))
                 (swap! mResult conj m)
                 (recur mResult [(vec (rest (get mTemp 0)))(vec (rest (get mTemp 1))) (vec (rest (get mTemp 2)))])
                 )
             )
        )
    )

#'user/transpuesta

In [123]:
(vec @(transpuesta matrixA))

[[1 8.9 91] [1 8.9 87] [1 8.9 130] [1 8.9 50] [1 8.9 89] [1 8.9 102] [1 8.9 75] [1 8.9 135] [1 9.0 90] [1 9.0 60] [1 9.0 152] [1 9.1 105] [1 9.1 56] [1 9.1 166] [1 9.3 132] [1 9.3 68] [1 9.3 60] [1 9.3 83] [1 9.3 90] [1 9.3 61]]

Multiplicador de matrices


In [14]:
(defn multiplicador [matrizA matrizB]
    (loop [mA mB mTemp () mResultado () contadorA 0 contadorB 0]
        (if (= (count mA) 0)
            
        )
    )
    
    
    
    )

Syntax error compiling at (REPL:2:5).
Unable to resolve symbol: mB in this context


class clojure.lang.Compiler$CompilerException: 